[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EN/quantlets/EMQA_arima_oil/EMQA_arima_oil.ipynb)

# EMQA_arima_oil

**Oil Price ARIMA Analysis**

Four-panel analysis of Brent crude oil prices: levels, log returns, ACF, and return distribution.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
from statsmodels.tsa.stattools import acf

brent = fetch('BZ=F')
log_ret = np.log(brent / brent.shift(1)).dropna()

fig, axes = plt.subplots(2, 2, figsize=(14, 9))

# Panel 1: Price levels
axes[0, 0].plot(brent.index, brent.values, color=COLORS['blue'], linewidth=0.8)
axes[0, 0].set_title('Brent Crude Oil — Price Levels', fontweight='bold')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Price (USD)')

# Panel 2: Log returns
axes[0, 1].plot(log_ret.index, log_ret.values, color=COLORS['red'], linewidth=0.5, alpha=0.8)
axes[0, 1].axhline(0, color=COLORS['gray'], linestyle='--', linewidth=0.5)
axes[0, 1].set_title('Log Returns', fontweight='bold')
axes[0, 1].set_xlabel('Date')
axes[0, 1].set_ylabel('Log Return')

# Panel 3: ACF of returns
nlags = 40
acf_vals = acf(log_ret, nlags=nlags)
axes[1, 0].bar(range(nlags + 1), acf_vals, width=0.4, color=COLORS['green'], alpha=0.85)
conf = 1.96 / np.sqrt(len(log_ret))
axes[1, 0].axhline(conf, color=COLORS['red'], linestyle='--', linewidth=0.8)
axes[1, 0].axhline(-conf, color=COLORS['red'], linestyle='--', linewidth=0.8)
axes[1, 0].axhline(0, color='black', linewidth=0.5)
axes[1, 0].set_title('ACF of Log Returns', fontweight='bold')
axes[1, 0].set_xlabel('Lag')
axes[1, 0].set_ylabel('ACF')

# Panel 4: Histogram with normal overlay
axes[1, 1].hist(log_ret, bins=60, density=True, color=COLORS['blue'], alpha=0.6, edgecolor='white')
x_range = np.linspace(log_ret.min(), log_ret.max(), 200)
mu, sigma = log_ret.mean(), log_ret.std()
from scipy.stats import norm
axes[1, 1].plot(x_range, norm.pdf(x_range, mu, sigma), color=COLORS['red'], linewidth=2,
                label='Normal fit')
axes[1, 1].set_title('Return Distribution', fontweight='bold')
axes[1, 1].set_xlabel('Log Return')
axes[1, 1].set_ylabel('Density')
axes[1, 1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False)

fig.suptitle('Brent Crude Oil — ARIMA Analysis', fontsize=14, fontweight='bold', y=1.02)
fig.tight_layout()
save_fig(fig, 'arima_oil_analysis.pdf')
plt.show()
